In [1]:
%run C:/Users/MohammedSB/Desktop/projects/Hypertension/Requirements.ipynb

In [2]:
# Seed
set_seed(0)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
PATH = r"D:\\MohammedSB\\Fundus\\"
CSV_PATH = {"HTNPath": PATH + r"HTN", "NonHTNPath": PATH + "NonHTN"}

MODELS_PATH = r"C:\Users\MohammedSB\Desktop\projects\Hypertension\models"

In [5]:
train_set, test_set = get_datasets()

In [6]:
BATCH_SIZE = 8

transform = T.Compose([
    T.Resize((512, 512)),
    T.ToTensor(),
    T.Normalize(mean=[0.3675, 0.1760, 0.1047], std=[0.2289, 0.1180, 0.0748]),
])

train_dataset = HypertensionDataset(CSV_PATH, train_set, train_transform=transform)
test_dataset = HypertensionDataset(CSV_PATH, test_set, test_transform=transform)

In [7]:
def calculate_results(N, trues, probs, threshold=0.5):
    f1_scores = []
    auc_scores = []
    pr_scores = []
    precision_scores = []
    recall_scores = []
    specificity_scores = []

    for i in range(N):
        fpr, tpr, _ = roc_curve(trues[i], probs[i])
        precision, recall, _ = precision_recall_curve(trues[i], probs[i])

        
        f1_scores.append(f1_score(trues[i], np.array(probs[i])>=threshold))
        auc_scores.append(auc(fpr, tpr))
        pr_scores.append(auc(recall, precision))
        precision_scores.append(precision_score(trues[i], np.array(probs[i])>=threshold))
        recall_scores.append(recall_score(trues[i], np.array(probs[i])>=threshold))
        specificity_scores.append(recall_score(trues[i], np.array(probs[i])>=threshold, pos_label=0))
    
    print(f"f1: {np.percentile(f1_scores, [2.5, 97.5])}") 
    print(f"auroc: {np.percentile(auc_scores, [2.5, 97.5])}")
    print(f"auprc: {np.percentile(pr_scores, [2.5, 97.5])}") 
    print(f"precision: {np.percentile(precision_scores, [2.5, 97.5])}") 
    print(f"recall: {np.percentile(recall_scores, [2.5, 97.5])}") 
    print(f"specificity: {np.percentile(specificity_scores, [2.5, 97.5])}")
    
    
def bootstrap(N, models, data_set, device, criterion=None, method="torch"):
    num_samples = len(data_set)
    trues, probs = dict(), dict()

    for i in tqdm(range(N)):

        if method == "torch":
            sampler = torch.utils.data.RandomSampler(data_set, replacement=True, num_samples=num_samples)
            data_loader = DataLoader(data_set, batch_size=8, sampler=sampler)

            metrics = test(models=models, criterion=criterion, test_loader=data_loader, device=device, show_output=False)
            y_true = metrics["y_true"].squeeze(1).numpy()
            y_prob = metrics["y_prob"].squeeze(1).numpy()

            trues[i] = y_true.tolist()
            probs[i] = y_prob.tolist()
        elif method == "sklearn":
            bootstrapped_data = resample(data_set, replace=True, n_samples=num_samples)
            x, y = bootstrapped_data[:, :-1], bootstrapped_data[:, -1]
            probas = boost.predict_proba(x)[:, 1]
            
            trues[i] = y.tolist()
            probs[i] = probas.tolist()
    
    return trues, probs


def bootstrap_session(N, models, data_set, device, criterion, show_results=True, method="torch", loc=None):
    
    results = dict()
    
    trues, probs = bootstrap(N, models, data_set, device, criterion, method=method)

    results["true"] = trues
    results["prob"] = probs
    
    trues, probs = [], []

    for i in range(N):
        trues += results["true"][i]
        probs += results["prob"][i]
    
    if show_results:
        print("Results:")
        calculate_results(N, results['true'], results['prob'], 0.5) 
    
    if loc is not None:
        with open(loc+".json", 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=4)
    else:
        return trues, probs

def optimal_threshold(trues, probs):
    precision, recall, thresholds = precision_recall_curve(trues, probs)
    fscore = (2 * precision * recall) / (precision + recall)
    ix = np.argmax(fscore)
    print('Best Threshold=%f, F-Score=%.3f' % (thresholds[ix], fscore[ix]))
    return thresholds[ix]

In [8]:
N=1000

FM_PATH = MODELS_PATH + r"\FundusModel.pth"
DM_PATH = MODELS_PATH + r"\DemographicFCNN.pth"

criterion = nn.BCEWithLogitsLoss()

# FundusModel Results

In [9]:
image_model = get_densenet201(device=device, freeze=True, with_mlp=True)
image_model.load_state_dict(torch.load(FM_PATH))

<All keys matched successfully>

In [10]:
models = {
    "image_model": image_model,
    "tabular_model": None,
    "fusion_model": None
}

In [11]:
bootstrap_session(N, models=models, data_set=test_dataset, device=device, criterion=criterion, loc='Results/FundusModel')

100%|██████████| 1000/1000 [2:02:32<00:00,  7.35s/it] 


Results:
f1: [0.6615     0.77359513]
auroc: [0.67111445 0.79618103]
auprc: [0.62529124 0.80026757]
precision: [0.51036811 0.647393  ]
recall: [0.91596442 0.99082569]
specificity: [0.23076923 0.39827434]


# DemographicModels Results

In [12]:
train_x, train_y = train_set[["Age", "Gender"]], train_set["Hypertension"]
test_x, test_y = test_set[["Age", "Gender"]], test_set["Hypertension"]

combined_test = np.hstack([test_x, np.expand_dims(test_y, -1)])

## XGBoost

In [13]:
with open(MODELS_PATH+'/DemographicXGBParams.json') as f:
    params = json.load(f)
    
boost = xgb.XGBClassifier(**params, objective="binary:logistic")

boost.fit(train_x, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.3,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.0001, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=3, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [14]:
bootstrap_session(N, models=boost, data_set=combined_test, device=device, criterion=criterion,\
                  method="sklearn", loc='Results/DemographicXGB')

100%|██████████| 1000/1000 [00:01<00:00, 750.39it/s]


Results:
f1: [0.67652817 0.79554492]
auroc: [0.67893514 0.80429802]
auprc: [0.53123468 0.76458057]
precision: [0.56521033 0.71796875]
recall: [0.80670025 0.92594292]
specificity: [0.42590278 0.61389947]


## SVM

In [15]:
with open(MODELS_PATH+'/DemographicSVMParams.json') as f:
    params = json.load(f)
    
svm = SVC(**params)

svm.fit(train_x, train_y)

SVC(C=1000, gamma=0.01)

In [16]:
bootstrap_session(N, models=svm, data_set=combined_test, device=device, criterion=criterion, \
                  method="sklearn", loc='Results/DemographicSVM')

100%|██████████| 1000/1000 [00:01<00:00, 758.58it/s]


Results:
f1: [0.67435976 0.78985507]
auroc: [0.67441774 0.80183051]
auprc: [0.52511126 0.76481834]
precision: [0.56081081 0.71523294]
recall: [0.80356411 0.92481673]
specificity: [0.43112713 0.60749805]


## FCNN

In [17]:
tabular_model = nn.Sequential(
    nn.Linear(in_features=2, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=32),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=32, out_features=16),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=16, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=1),
)

tabular_model = tabular_model.to(device)
tabular_model.load_state_dict(torch.load(DM_PATH))

<All keys matched successfully>

In [18]:
models = {
    "image_model": None,
    "tabular_model": tabular_model,
    "fusion_model": None
}

In [19]:
bootstrap_session(N, models=models, data_set=test_dataset, device=device, criterion=criterion, loc='Results/DemographicFCNN')

100%|██████████| 1000/1000 [1:16:36<00:00,  4.60s/it]


Results:
f1: [0.67703981 0.79054447]
auroc: [0.69891585 0.81851042]
auprc: [0.64978408 0.81064488]
precision: [0.55487805 0.69812833]
recall: [0.83177339 0.94644042]
specificity: [0.38786391 0.5714591 ]


# FusionModel Results

## CombineFeatures

In [20]:
tm_path = MODELS_PATH + r"\CombineFeatures\DemographicPath.pth"
img_path = MODELS_PATH + r"\CombineFeatures\FundusPath.pth"
fm_path = MODELS_PATH + r"\CombineFeatures\FusionPath.pth"


tabular_model = nn.Sequential(
    nn.Linear(in_features=2, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=32),
)

image_model = get_densenet201(device=device, freeze=True, with_mlp=False, outputs=32)

fusion_model = nn.Sequential(
    nn.Linear(in_features=64, out_features=128),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=128, out_features=32),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=32, out_features=16),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=16, out_features=1),
)

tabular_model, image_model, fusion_model = tabular_model.to(device).float(), image_model.to(device).float(),\
                                                fusion_model.to(device).float()

tabular_model.load_state_dict(torch.load(tm_path))
image_model.load_state_dict(torch.load(img_path))
fusion_model.load_state_dict(torch.load(fm_path))

<All keys matched successfully>

In [21]:
models = {
    "image_model": image_model,
    "tabular_model": tabular_model,
    "fusion_model": fusion_model,
}

In [22]:
bootstrap_session(N, models=models, data_set=test_dataset, device=device, criterion=criterion, loc='Results/CombineFeatures')

100%|██████████| 1000/1000 [2:01:57<00:00,  7.32s/it] 


Results:
f1: [0.66922598 0.78723404]
auroc: [0.7205815  0.83794064]
auprc: [0.66107787 0.83009923]
precision: [0.54164701 0.68824498]
recall: [0.84402953 0.95412844]
specificity: [0.35041667 0.53508772]


## CombineOutputs

In [23]:
tm_path = MODELS_PATH + r"\CombineOutputs\DemographicPath.pth"
img_path = MODELS_PATH + r"\CombineOutputs\FundusPath.pth"
fm_path = MODELS_PATH + r"\CombineOutputs\FusionPath.pth"

tabular_model = nn.Sequential(
    nn.Linear(in_features=2, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=32),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=32, out_features=16),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=16, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=1),
)

image_model = get_densenet201(device=device, freeze=True, with_mlp=True, outputs=1)

fusion_model = nn.Sequential(
    nn.Linear(in_features=2, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=16),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=16, out_features=4),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=4, out_features=1),
)

tabular_model, image_model, fusion_model = tabular_model.to(device).float(), image_model.to(device).float(),\
                                                fusion_model.to(device).float()

tabular_model.load_state_dict(torch.load(tm_path))
image_model.load_state_dict(torch.load(img_path))
fusion_model.load_state_dict(torch.load(fm_path))

<All keys matched successfully>

In [24]:
models = {
    "image_model": image_model,
    "tabular_model": tabular_model,
    "fusion_model": fusion_model,
}

In [25]:
bootstrap_session(N, models=models, data_set=test_dataset, device=device, criterion=criterion, loc='Results/CombineOutputs')

100%|██████████| 1000/1000 [2:02:03<00:00,  7.32s/it] 


Results:
f1: [0.66382979 0.78791362]
auroc: [0.72139707 0.83419071]
auprc: [0.65764018 0.8358386 ]
precision: [0.57040038 0.72222222]
recall: [0.77226854 0.90656369]
specificity: [0.45967197 0.63493547]


## KeepFeatures

In [26]:
image_model = get_densenet201(device=device, freeze=True, with_mlp=True)
image_model.load_state_dict(torch.load(FM_PATH))

image_model = image_model.to(device).float()

models = {
    "image_model": image_model,
    "tabular_model": None,
    "fusion_model": None,
}

In [27]:
train_x, train_y = build_tabular_dataset(models, train_dataset, method="keep_features")

test_x, test_y = build_tabular_dataset(models, test_dataset, method="keep_features")
test_fusion_set = InputOutputDataset(test_x, test_y)

combined_test = np.hstack([test_x, np.expand_dims(test_y, -1)])

## XGBoost

In [28]:
with open(MODELS_PATH+ '/KeepFeatures/XGBParams.json') as f:
    params = json.load(f)
    
boost = xgb.XGBClassifier(**params, objective="binary:logistic")

boost.fit(train_x, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.7,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.005, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=3, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=1000, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [29]:
bootstrap_session(N, models=boost, data_set=combined_test, device=device, criterion=criterion,\
                  method="sklearn", loc='Results/KeepFeaturesXGB')

100%|██████████| 1000/1000 [00:01<00:00, 556.72it/s]


Results:
f1: [0.63114663 0.76116187]
auroc: [0.6397735  0.77288224]
auprc: [0.55877038 0.76210067]
precision: [0.54725123 0.71853501]
recall: [0.70243872 0.85454934]
specificity: [0.46210664 0.643493  ]


## SVM

In [30]:
with open(MODELS_PATH+ '/KeepFeatures/SVMParams.json') as f:
    params = json.load(f)
        
svm = SVC(**params)

svm.fit(train_x, train_y)

SVC(C=1000, gamma=0.01)

In [31]:
bootstrap_session(N, models=svm, data_set=combined_test, device=device, criterion=criterion,\
                  method="sklearn", loc='Results/KeepFeaturesSVM')

100%|██████████| 1000/1000 [00:01<00:00, 563.91it/s]


Results:
f1: [0.63708864 0.75410711]
auroc: [0.64403184 0.77217561]
auprc: [0.56759036 0.75584324]
precision: [0.56023008 0.71094575]
recall: [0.70191886 0.8500463 ]
specificity: [0.47271465 0.64547203]


## FCNN

In [32]:
fm_path = MODELS_PATH + r"\KeepFeatures\FusionPath.pth"

fusion_model = nn.Sequential(
    nn.Linear(in_features=3, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=32),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=32, out_features=16),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=16, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=1),
)

fusion_model = fusion_model.to(device).float()
fusion_model.load_state_dict(torch.load(fm_path))

<All keys matched successfully>

In [33]:
models = {
    "image_model": None,
    "tabular_model": None,
    "fusion_model": fusion_model
}

In [34]:
bootstrap_session(N, models=models, data_set=test_fusion_set, device=device, criterion=criterion,\
                  loc='Results/KeepFeaturesFCNN')

100%|██████████| 1000/1000 [00:36<00:00, 27.65it/s]


Results:
f1: [0.67692015 0.789143  ]
auroc: [0.68774533 0.81447842]
auprc: [0.6427793  0.81375371]
precision: [0.55553763 0.70069573]
recall: [0.82856421 0.94392523]
specificity: [0.40350509 0.57549932]


# VotingFeatures

In [35]:
# Load image and Tabular model
image_model = get_densenet201(device=device, freeze=True, with_mlp=True, outputs=1)

tabular_model = nn.Sequential(
    nn.Linear(in_features=2, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=32),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=32, out_features=16),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=16, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=1),
)

image_model.load_state_dict(torch.load(FM_PATH))
tabular_model.load_state_dict(torch.load(DM_PATH))

image_model, tabular_model = image_model.to(device).float(), tabular_model.to(device).float()

# load fusion model
tabular_path = nn.Sequential(
    nn.Linear(in_features=2, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=32),
)

image_path = get_densenet201(device=device, freeze=True, with_mlp=False, outputs=32)

fusion_path = nn.Sequential(
    nn.Linear(in_features=64, out_features=128),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=128, out_features=32),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=32, out_features=16),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=16, out_features=1),
)

image_path.load_state_dict(torch.load(MODELS_PATH + "\CombineFeatures\FundusPath.pth"))
tabular_path.load_state_dict(torch.load(MODELS_PATH + "\CombineFeatures\DemographicPath.pth"))
fusion_path.load_state_dict(torch.load(MODELS_PATH + "\CombineFeatures\FusionPath.pth"))

tabular_path, image_path, fusion_path = tabular_path.to(device).float(), image_path.to(device).float(),\
                                                fusion_path.to(device).float()

fusion_model = dict()
fusion_model["image_path"], fusion_model["tabular_path"], fusion_model["fusion_path"] = image_path, \
                                                                                    tabular_path, fusion_path

In [36]:
models = {
    "image_model": image_model,
    "tabular_model": tabular_model,
    "fusion_model": fusion_model,
}

In [37]:
train_x, train_y = build_tabular_dataset(models, train_dataset, method="voting_features")

test_x, test_y = build_tabular_dataset(models, test_dataset, method="voting_features")
test_fusion_set = InputOutputDataset(test_x, test_y)

combined_test = np.hstack([test_x, np.expand_dims(test_y, -1)])

## XGBoost

In [38]:
with open(MODELS_PATH+'/VotingFeatures/XGBParams.json') as f:
    params = json.load(f)
    
boost = xgb.XGBClassifier(**params, objective="binary:logistic")

boost.fit(train_x, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.3,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.005, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=3, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=500, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [39]:
bootstrap_session(N, models=boost, data_set=combined_test, device=device, criterion=criterion,\
                  method="sklearn", loc='Results/VotingFeaturesXGB')

100%|██████████| 1000/1000 [00:01<00:00, 610.14it/s]


Results:
f1: [0.66406094 0.78549676]
auroc: [0.70219437 0.82188266]
auprc: [0.67925741 0.83085986]
precision: [0.5443038  0.69187165]
recall: [0.8204955  0.94285714]
specificity: [0.38278764 0.55557971]


## SVM

In [40]:
with open(MODELS_PATH+ '/KeepFeatures/SVMParams.json') as f:
    params = json.load(f)
        
svm = SVC(**params)

svm.fit(train_x, train_y)

SVC(C=1000, gamma=0.01)

In [41]:
bootstrap_session(N, models=svm, data_set=combined_test, device=device, criterion=criterion,\
                  method="sklearn", loc='Results/VotingFeaturesSVM')

100%|██████████| 1000/1000 [00:01<00:00, 601.83it/s]


Results:
f1: [0.66423221 0.78447735]
auroc: [0.70300574 0.82762664]
auprc: [0.67684662 0.83412412]
precision: [0.5443038  0.70001497]
recall: [0.82257688 0.94059616]
specificity: [0.378767   0.56757329]


## FCNN

In [42]:
fm_path = MODELS_PATH + "\VotingFeatures\FusionPath.pth"

fusion_model = nn.Sequential(
    nn.Linear(in_features=3, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=32),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=32, out_features=16),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=16, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=1),
)

fusion_model = fusion_model.to(device).float()
fusion_model.load_state_dict(torch.load(fm_path))

<All keys matched successfully>

In [43]:
models = {
    "image_model": None,
    "tabular_model": None,
    "fusion_model": fusion_model
}

In [44]:
bootstrap_session(N, models=models, data_set=test_fusion_set, device=device, criterion=criterion,\
                  loc='Results/VotingFeaturesFCNN')

100%|██████████| 1000/1000 [00:36<00:00, 27.24it/s]


Results:
f1: [0.66666667 0.78351885]
auroc: [0.66813748 0.79278959]
auprc: [0.59462648 0.78277043]
precision: [0.53888841 0.68629556]
recall: [0.83783206 0.94599295]
specificity: [0.35714286 0.54241005]


## Ensemble

In [45]:
sig = nn.Sigmoid()

tensored_x, tensored_y = torch.tensor(test_x), torch.tensor(test_y)
probs = sig(tensored_x)

average = torch.mean(probs, axis=1)

esnemble_average = torch.stack([average, tensored_y], axis=-1).numpy()

In [46]:
trues, probs = dict(), dict()
results = dict()

for i in tqdm(range(N)):
    bootstrapped_data = resample(esnemble_average, replace=True, n_samples=len(esnemble_average))
    x, y = bootstrapped_data[:, :-1], bootstrapped_data[:, -1]
    
    trues[i] = y.tolist()
    probs[i] = x.tolist()
    
results["true"] = trues
results["prob"] = probs

100%|██████████| 1000/1000 [00:00<00:00, 4784.96it/s]


In [47]:
with open('Results/VotingFeaturesEnsemble.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

In [50]:
trues, probs = [], []

for i in range(N):
    trues += results["true"][i]
    probs += results["prob"][i]
    
calculate_results(N, results['true'], results['prob'], 0.5) 

100%|██████████| 1000/1000 [00:00<00:00, 133934.86it/s]


f1: [0.66411908 0.78500122]
auroc: [0.64482753 0.7766236 ]
auprc: [0.5816743  0.77394475]
precision: [0.53626981 0.68837662]
recall: [0.83193091 0.94646351]
specificity: [0.3627451  0.54205607]
